In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import math

# Import the backtrader platform
import backtrader as bt

from sklearn.model_selection import GridSearchCV
from sklearn import svm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# SVR

In [2]:
# Create a Stratey
class SVRStrategy(bt.Strategy):
    
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataclose_spx = self.datas[1].close

        
        # Add indicators
        self.ema_short = bt.indicators.ExponentialMovingAverage(self.datas[1], period=12)
        self.ema_long = bt.indicators.ExponentialMovingAverage(self.datas[1], period=26)
        
        # Add X variables
        self.dema = self.ema_short - self.ema_long
        self.mom = bt.Cmp(self.ema_short, self.ema_long)
        
        self.rsi_overbought = bt.indicators.RSI(self.datas[1]) > 70
        self.rsi_oversold = bt.indicators.RSI(self.datas[1]) < 30
        
        self.cross_over = bt.indicators.CrossOver(self.ema_short, self.ema_long) 
        
        #self.smma = bt.indicators.SmoothedMovingAverage(self.rsi, period=10)
        #self.atr = bt.indicators.ATR(self.datas[0], plot=False)
        
        # Compute daily return
        self.vix_returns = self.dataclose / self.dataclose(-1) - 1
        

    def next(self):
        
        lag = 1
        # Set the length of training set
        trainsize = 252
        count = len(self.dataclose) - 26
        
        # Simply log the closing price of the series from the reference

        self.log('VIX Close, %.4f' % self.dataclose[0])

        
        # Regression
        
        trainX1 = pd.DataFrame(self.mom.get(ago=-lag, size=trainsize).tolist())
        trainX2 = pd.DataFrame(self.rsi_overbought.get(ago=-lag, size=trainsize).tolist())
        trainX3 = pd.DataFrame(self.rsi_oversold.get(ago=-lag, size=trainsize).tolist())
        trainX4 = pd.DataFrame(self.cross_over.get(ago=-lag, size=trainsize).tolist())
        trainX5 = pd.DataFrame(self.dataclose.get(ago=-lag, size=trainsize).tolist())
        
        trainX = pd.concat([trainX1, trainX2, trainX3, trainX4, trainX5], axis=1)
        #train_scaled = preprocessing.scale(trainX)

        
        trainY = np.array(self.dataclose.get(ago=0, size=trainsize).tolist()).reshape(-1, 1)
        
        testX = np.array([self.mom[1-lag], self.rsi_overbought[1-lag], self.rsi_oversold[1-lag], self.cross_over[1-lag], self.dataclose[1-lag]]).reshape(-1,5)
        
        pred = []
        
        if count <= trainsize + lag - 1:
            
            # Set the predicted closeprice to the previous one if no regression is running
            pred.append(self.dataclose[0])
            
        else:
            
            regr = svm.SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
            regr.fit(trainX, trainY.ravel())
            tmp_pred = regr.predict(testX)
            pred.append(tmp_pred) # use current data to predict
            
#         print('Predicted return, %.4f' % r_pred_rf[-1])
#         print('Mean Squared Error, %.4f' % np.nanmean(mse))
         
        if pred[-1] > self.dataclose[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        if pred[-1] < self.dataclose[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
            self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
            self.order = self.sell()

In [3]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(SVRStrategy)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mySharpe')

    datapath_spx = os.path.join('../../../datas/spx-2013-2018.txt')
    datapath_vix= os.path.join('../../../datas/vix-2013-2018.txt')

    # Create a Data Feed
    data_vix = bt.feeds.YahooFinanceCSVData(
        dataname=datapath_vix,
        # Do not pass values before this date
        #fromdate=datetime.datetime(2018, 1, 1),
        # Do not pass values before this date
        #todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    data_spx = bt.feeds.YahooFinanceCSVData(
        dataname=datapath_spx,
        # Do not pass values before this date
        #fromdate=datetime.datetime(2018, 1, 1),
        # Do not pass values before this date
        #todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data_vix)
    cerebro.adddata(data_spx)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='0315_test.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    #print('Holding Period Return:', thestrat.analyzers.myHPY.get_analysis())

Starting Portfolio Value: 100000.00
2013-03-19, VIX Close, 14.3900
2013-03-20, VIX Close, 12.6700
2013-03-21, VIX Close, 13.9900
2013-03-22, VIX Close, 13.5700
2013-03-25, VIX Close, 13.7400
2013-03-26, VIX Close, 12.7700
2013-03-27, VIX Close, 13.1500
2013-03-28, VIX Close, 12.7000
2013-04-01, VIX Close, 13.5800
2013-04-02, VIX Close, 12.7800
2013-04-03, VIX Close, 14.2100
2013-04-04, VIX Close, 13.8900
2013-04-05, VIX Close, 13.9200
2013-04-08, VIX Close, 13.1900
2013-04-09, VIX Close, 12.8400
2013-04-10, VIX Close, 12.3600
2013-04-11, VIX Close, 12.2400
2013-04-12, VIX Close, 12.0600
2013-04-15, VIX Close, 17.2700
2013-04-16, VIX Close, 13.9600
2013-04-17, VIX Close, 16.5100
2013-04-18, VIX Close, 17.5600
2013-04-19, VIX Close, 14.9700
2013-04-22, VIX Close, 14.3900
2013-04-23, VIX Close, 13.4800
2013-04-24, VIX Close, 13.6100
2013-04-25, VIX Close, 13.6200
2013-04-26, VIX Close, 13.6100
2013-04-29, VIX Close, 13.7100
2013-04-30, VIX Close, 13.5200
2013-05-01, VIX Close, 14.4900
201

This strategy does not work...!